In [1]:
import pandas as pd
import numpy as np
import requests
import json
import datetime
from tvDatafeed import TvDatafeed, Interval

In [2]:
def getdata(name):
    data=tv.get_hist(symbol=name,exchange='BINANCE',interval=Interval.in_daily,n_bars=5000)
    data=data.reset_index()
    data["datetime"]=data["datetime"].dt.strftime('%Y-%m-%d')
    data2=data[data["datetime"]>="2020-01-01"].copy()
    data2["datetime"] = pd.to_datetime(data2["datetime"])
    return data2

In [3]:
def getapi(api):
    response_API=requests.get(api)
    data=response_API.text
    parse_json=json.loads(data)
    df = pd.json_normalize(parse_json['values'])
    listtime=list(df['x'])
    listtime2=[]
    for i in listtime:
        listtime2.append(datetime.datetime.fromtimestamp(i).strftime('%Y-%m-%d'))
    df['datetime']=pd.Series(listtime2)
    df["datetime"] = pd.to_datetime(df["datetime"])
    df=df[df["datetime"]>="2020-01-01"].copy()
    df=df.drop(columns=['x'])
    df.rename(columns={'y':'value'},inplace=True)
    df=df.drop_duplicates(subset=['datetime'])
    return df

In [4]:
tv = TvDatafeed()
bitcoi=getdata('BTCUSDT')
eth=getdata('ETHUSDT')
bnb=getdata('BNBUSDT')
ada=getdata('ADAUSDT')
bitcoi=bitcoi[["datetime","open","close"]]
bitcoi.tail(5)

you are using nologin method, data you access may be limited


,datetime,open,close
1654,2022-02-25,39219.16,39116.72
1655,2022-02-26,39116.73,37699.07
1656,2022-02-27,37699.08,43160.00
1657,2022-02-28,43160.00,44421.20
1658,2022-03-01,44421.20,43706.16


In [5]:
number_transactions=getapi('https://api.blockchain.info/charts/n-transactions?timespan=4years&rollingAverage=24hours&format=json&sampled=false')
number_address=getapi('https://api.blockchain.info/charts/n-unique-addresses?timespan=4years&rollingAverage=24hours&format=json&sampled=false')
transaction_second=getapi('https://api.blockchain.info/charts/transactions-per-second?timespan=4years&rollingAverage=24hours&format=json&sampled=false')
transaction_second=transaction_second.groupby(['datetime']).mean().reset_index()
total_bitcoin=getapi('https://api.blockchain.info/charts/total-bitcoins?timespan=4years&rollingAverage=24hours&format=json&sampled=true')
hash_rate=getapi('https://api.blockchain.info/charts/hash-rate?timespan=4years&rollingAverage=24hours&format=json&sampled=false')

In [6]:
bitcoin=bitcoi.merge(eth[["datetime","open"]],on='datetime',how='left',suffixes=('_bitcoin', '_eth'))
bitcoin=bitcoin.merge(bnb[["datetime","open"]],on='datetime',how='left')
bitcoin=bitcoin.merge(ada[["datetime","open"]],on='datetime',how='left',suffixes=('_bnb', '_ada'))

In [7]:
data_api=number_transactions.merge(number_address,on='datetime',how='left',suffixes=('_number_transaction', '_number_address'))
data_api=data_api.merge(transaction_second,on='datetime',how='left')
data_api=data_api.merge(total_bitcoin,on='datetime',how='left',suffixes=('_transaction_second', '_total_bitcoin'))
data_api=data_api.merge(hash_rate,on='datetime',how='left')
data_api.rename(columns={'value':'value_hash_rate'},inplace=True)

In [8]:
bitcoin=bitcoin.merge(data_api,on='datetime',how='left')
bitcoin.rename(columns={'close':'close_bitcoin'},inplace=True)
bitcoin.tail(5)

,datetime,open_bitcoin,close_bitcoin,open_eth,open_bnb,open_ada,value_number_transaction,value_number_address,value_transaction_second,value_total_bitcoin,value_hash_rate
786,2022-02-25,39219.16,39116.72,2768.05,375.0,0.898,220423.0,582374.0,2.964757,1.896703e+07,1.737818e+08
787,2022-02-26,39116.73,37699.07,2779.47,373.7,0.886,211164.0,583361.0,2.979167,1.896793e+07,1.890746e+08
788,2022-02-27,37699.08,43160.00,2616.79,360.1,0.857,270918.0,702483.0,2.541493,1.896882e+07,2.140992e+08
789,2022-02-28,43160.00,44421.20,2920.83,395.7,0.962,280634.0,722863.0,2.425868,1.896972e+07,2.252213e+08
790,2022-03-01,44421.20,43706.16,2976.33,408.7,0.964,NaN,NaN,NaN,NaN,NaN


In [9]:
bitcoin.to_pickle("../data/interim/bitcoin_value_2020-now.pkl")